<a href="https://colab.research.google.com/github/MathMachado/DSWP/blob/master/Notebooks/NB10_04__3DP_3_Data_Transformation.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

<center><h1><b><i>3DP_3 - DATA TRANSFORMATION</i></b></h1></center>

* **Objetivo**: Preparar os dados para o Machine Learning.

# **AGENDA**:

> Consulte **Table of contents**.


# **Melhorias da sessão**
* Desenvolver a sessão sobe WOE.

___
# **Referências**
* [Why, How and When to Scale your Features](https://medium.com/greyatom/why-how-and-when-to-scale-your-features-4b30ab09db5e)
* [Demonstrating the different strategies of KBinsDiscretizer](https://scikit-learn.org/stable/auto_examples/preprocessing/plot_discretization_strategies.html#sphx-glr-auto-examples-preprocessing-plot-discretization-strategies-py);
* [Why do we need feature scaling in Machine Learning and how to do it using SciKit Learn?](https://medium.com/@contactsunny/why-do-we-need-feature-scaling-in-machine-learning-and-how-to-do-it-using-scikit-learn-d8314206fe73)
* [Importance of Feature Scaling](https://scikit-learn.org/stable/auto_examples/preprocessing/plot_scaling_importance.html#sphx-glr-auto-examples-preprocessing-plot-scaling-importance-py) --> Muito importante por demonstrar os efeitos e a importância de se transformar as colunas numéricas.
* [Feature discretization](https://scikit-learn.org/stable/auto_examples/preprocessing/plot_discretization_classification.html#sphx-glr-auto-examples-preprocessing-plot-discretization-classification-py) --> Mostra o impacto na acurácia dos modelos com e sem discretização. Ou seja, discretizar faz sentido!

___
# **Machine Learning com Python (Scikit-Learn)**

![Scikit-Learn](https://github.com/MathMachado/Materials/blob/master/scikit-learn-1.png?raw=true)

# Porque dimensionar (Scale), padronizar (Standardize) e normalizar (Normalize) importa?
* Porque muitos algoritmos de **Machine Learning** performam melhor ou convergem mais rápido quando os atributos/colunas/variáveis estão na mesma escala e possuem distribuição "próxima" da Normal.

## Carregar as bibliotecas (genéricas) Python

In [ ]:
!pip install category_encoders
!pip install update

In [ ]:
import pandas as pd

import numpy as np
from sklearn import preprocessing
import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline

import category_encoders as ce # library para aplicação do WOE - Weight Of Evidence para avaliar importância dos atributos

# remove warnings to keep notebook clean
import warnings
warnings.filterwarnings('ignore')

In [ ]:
pd.options.display.float_format = '{:.2f}'.format

## Carregar os dados

### Dataframe gerado aleatoriamente - variáveis com distribuição Normal

In [ ]:
np.random.seed(20111974)

i_N = 10000

df_A1 = pd.DataFrame({
    'coluna1': np.random.normal(0, 2, i_N), # Observem que a média das colunas são distintas
    'coluna2': np.random.normal(50, 3, i_N),
    'coluna3': np.random.normal(-5, 5, i_N),
    'coluna4': np.random.normal(-10, 10, i_N)
})

df_A1.head()

**Dica**: Podemos usar outras distribuições (se quisermos), como a Exponential (mostrada abaixo).

In [ ]:
np.random.seed(20111974)

df_A2 = pd.DataFrame({
    'coluna1': np.random.normal(0, 2, i_N),
    'coluna2': np.random.normal(50, 3, i_N),
    'coluna3': np.random.exponential(5, i_N), # coluna3 tem distribuição Exponential
    'coluna4': np.random.normal(-10, 10, i_N)
})

df_A2.head()

### Dataframe gerado aleatoriamente 2

In [ ]:
from sklearn.datasets import make_classification

dados, classe = make_classification(n_samples = i_N, n_features = 4, n_informative = 3, n_redundant = 1, n_classes = 3)

df_A3 = pd.DataFrame({'coluna1': dados[:,0],
                                  'coluna2':dados[:,1],
                                  'coluna3':dados[:,2],
                                  'coluna4':dados[:,3]}) #, 'coluna5':classe})

df_A3.head()

In [ ]:
df_A4 = pd.DataFrame({ 
    'coluna1': np.random.beta(5, 1, i_N) * 25, 
    'coluna2': np.random.exponential(5, i_N),
    'coluna3': np.random.normal(10, 2, i_N),
    'coluna4': np.random.normal(10, 10, i_N), 
})

df_A4.head()

#### Extração de amostras para compararmos

In [ ]:
df_A1_test = df_A1.sample(n = 100)
df_A2_test = df_A2.sample(n = 100)
df_A3_test = df_A3.sample(n = 100)
df_A4_test = df_A4.sample(n = 100)

___
# **Transformações**

## (1) StandardScaler
* StandardScaler é a transformação que centraliza os dados através da remoção da média (dos dados) e, na sequência, redimensiona (scale) através da divisão pelo desvio-padrão;
* Após a transformação, os dados terão média zero e desvio-padrão 1;
* **Assume que os dados (as colunas a serem transformadas) são normalmente distribuidos**;
* Se os dados não possuem distribuição Normal, então esta **NÃO** é uma boa transformação a se aplicar.

$$z_{i}= \frac{x_{i}-mean(x)}{std(x)}$$

### Exemplo

In [ ]:
df_A3.head()

Histograma:

In [ ]:
plt.figure(figsize = (12, 8))
plt.hist(df_A1['coluna3'], color = 'blue', edgecolor = 'black', bins = int(180/5))

# Adiciona títulos e labels
plt.title('Coluna3 - Distribuição Normal')

In [ ]:
plt.figure(figsize = (12, 8))
plt.hist(df_A2['coluna3'], color = 'blue', edgecolor = 'black', bins = int(180/5))

# Adiciona títulos e labels
plt.title('Coluna3 - Distribuição Exponencial')

Considere o gráfico a seguir:

In [ ]:
df_A1.plot(kind = 'kde') # KDE (= kernel Density Estimate) ajuda-nos a visualizar a distribuição dos dados, análogo ao histograma.

Qual a interpretação para o gráfico acima?

In [ ]:
df_A1.plot()

A seguir, a transformação StandardScaler:

In [ ]:
from sklearn.preprocessing import StandardScaler

O ideal é termos um array com as preditoras, da seguinte forma:
X = [coluna1, coluna2, ..., colunaN]

In [ ]:
np.set_printoptions(precision = 3)

A1_scale = StandardScaler().fit_transform(df_A1) # Combinação dos métodos fit() + transform()

A1_scale_fit = StandardScaler().fit(df_A1) # Aplica o fit() separadamente
A1_scale_transform = A1_scale_fit.transform(df_A1) # Aplica o transform() separadamente.
A1_scale_fit_transform = StandardScaler().fit(df_A1).transform(df_A1) # Aplica fit().transform() encadeado

A2_scale = StandardScaler().fit_transform(df_A2)

A3_scale = StandardScaler().fit_transform(df_A3)

## Salvar os parâmetros do StandardScaler e outros --> Colocar aqui!

In [ ]:
A1_scale_fit.scale_

Observe abaixo que A1_scale = A1_scale_transform = A1_scale_fit_transform --> São arrays multidimensionais (do tipo NumPy)!

In [ ]:
A1_scale

In [ ]:
A1_scale_transform

In [ ]:
A1_scale_fit_transform

Transformando em dataframe:

In [ ]:
df_A1_scale = pd.DataFrame(A1_scale, columns = ['coluna1', 'coluna2', 'coluna3', 'coluna4'])
df_A2_scale = pd.DataFrame(A2_scale, columns = ['coluna1', 'coluna2', 'coluna3', 'coluna4'])
df_A3_scale = pd.DataFrame(A3_scale, columns = ['coluna1', 'coluna2', 'coluna3', 'coluna4'])

Agora compare esse novo gráfico abaixo --> Vemos que os dados transformados tem distribuição Normal(0, 1):

In [ ]:
df_A1.head()

In [ ]:
df_A1_scale.head()

In [ ]:
df_A1_scale.plot(kind = 'kde')

In [ ]:
df_A2.plot(kind = 'kde')

In [ ]:
df_A2_scale.plot(kind = 'kde')

In [ ]:
df_A3.plot(kind = 'kde')

In [ ]:
df_A3_scale.plot(kind = 'kde')

### Exercício: Calcular a média e o desvio-padrão.

In [ ]:
df_A1.describe()

In [ ]:
df_A1_scale.describe()

#### Correlação das colunas
* Observe que as correlações entre as variáveis não se alteram com as transformações.

In [ ]:
df_A1.corr()

In [ ]:
df_A1_scale.corr()

Qual a conclusão?

## (2) MinMaxScaler
* **Transformação muito popular e utilizada**.
* Transforma os dados para o intervalo [0, 1];
* Se StandardScaler não é aplicável, então essa transformação funciona bem.
* Sensível aos _outliers_. Portanto, o ideal é que os _outliers_ sejam tratados previamente.
* Uma transformação similar à MinMaxScaler() é MaxAbsScaler() (redimensiona os dados no intervalo [-1, 1]) e centralizado em 0).
* Não corrige skewness;
* Sensível à outliers;

$$z_{i}= \frac{x_{i}-min(x)}{max(x)-min(x)}$$

### Exemplo

In [ ]:
from sklearn.preprocessing import MinMaxScaler

In [ ]:
df_A1.plot(kind = 'kde')

In [ ]:
A1_MinMaxScaler = MinMaxScaler().fit_transform(df_A1)
df_A1_MinMaxScaler = pd.DataFrame(A1_MinMaxScaler,columns = ['coluna1', 'coluna2', 'coluna3', 'coluna4'])

# Gráfico
df_A1_MinMaxScaler.plot(kind = 'kde')

Qual a conclusão?

## (3) RobustScaler
* Transformação ideal para dados com **outliers**.

$$z_{i}= \frac{x_{i}-Q_{1}(x)}{Q_{3}(x)-Q_{1}(x)}$$

In [ ]:
df_A1.plot(kind = 'kde')

In [ ]:
from sklearn.preprocessing import RobustScaler

In [ ]:
A1_RobustScaler = RobustScaler().fit_transform(df_A1)
df_A1_RobustScaler = pd.DataFrame(A1_RobustScaler, columns = ['coluna1', 'coluna2', 'coluna3', 'coluna4'])

# Gráfico
df_A1_RobustScaler.plot(kind = 'kde')

### **Insight**: Gerar aleatoriamente colunas/variáveis com distribuição Gamma, Beta, Normal, Exponential e etc e avaliar o impacto das várias transformações.

# **Wrap Up**
* Use MinMaxScaler como transformação default, pois esta transformação não distorce os dados;
* Use RobustScaler se seus dados/coluna/variável possui **outliers** e gostaríamos de reduzir o efeito/impacto destes **outliers**. Entretanto, o melhor tratamento é estudar os **outliers** cuidadosamente e tratá-los adequadamente;
* Use StandardScaler se seus dados/colunas/variáveis possuem distribuição Normal (ou pelo menos se aproxima bem da distribuição Normal).

## Encoding Variáveis Categóricas

### Encoding Variáveis Ordinais
* Exemplo: Variáveis com valores ordinais: baixo, médio ou alto.

#### Dataframe-exemplo:

In [ ]:
# Aqui vou usar a função randint - Retorna números inteiros aleatórios incluindo o número inferior e excluindo o superior.

l_idade = [
           np.random.randint(20, 40),
           np.random.randint(20, 40),
           np.random.randint(20, 40), 
           np.random.randint(20, 40),
           np.random.randint(20, 40),
           np.random.randint(20, 40),
           np.random.randint(20, 40),
           np.random.randint(20, 40),
           np.random.randint(20, 40),
           np.random.randint(20, 40)
          ]

l_salario = ['baixo', 'medio', 'alto']
l_salario2 = np.random.choice(l_salario, 10, p = [0.6, 0.3, 0.1])

df_A5 = pd.DataFrame({
    'idade': l_idade,
    'salario': l_salario2})

In [ ]:
df_A5

Neste exemplo, vamos redefinir a variável categórical ordinal 'Salario' da seguinte forma:

In [ ]:
df_A5['salario_cat'] = df_A5['salario'].map({'baixo': 1, 'medio': 2, 'alto': 3})
df_A5

### Encoding Variáveis Nominais
* Exemplo: Variáveis com valores nominais: Sexo (Feminino, Masculino).

* Use One-Hot Encoding ou pd.get.dummies()

Vamos utilizar o dataframe criado no passo anterior:

In [ ]:
df_A5['salario'].unique()

In [ ]:
from sklearn.preprocessing import LabelEncoder, OneHotEncoder

#### Aplicar LabelEncoder()

In [ ]:
le = LabelEncoder()
df_A5['salario_le'] = le.fit_transform(df_A5['salario'])
df_A5

In [ ]:
df_A5['salario'].value_counts()

#### Aplicar pd.get.dummies()

In [ ]:
dummies = pd.get_dummies(df_A5['salario'])
df_A5 = pd.concat([df_A5, dummies], axis = 1)
df_A5

# Power Transformations
* Tem por objetivo transformar a distribuição de probabilidade da variável/coluna a fim de torná-la Normal. Esta normalização é feita através da correção da skewness (estabilização da variância) da distribuição.
* Exemplos de Power Transformations:
    * log;
        * Ajuda com distribuições skewness;
        * Útil para distribuições não-negativas e sem zeros;
    * raiz quadrada;
    * raiz cúbica;
    * **Transformação de Box-Cox** e
    * Transformação de Yeo-Johson.

### Transformação de Yeo-Johnson (transformação default da librarie PowerTransformer)

In [ ]:
plt.figure(figsize = (12, 8))
plt.hist(df_A2['coluna3'], color = 'blue', edgecolor = 'black', bins = int(180/5))

# Adiciona títulos e labels
plt.title('Histograma da coluna3 - Distribuição Exponencial')

In [ ]:
from sklearn.preprocessing import PowerTransformer

In [ ]:
df_A2.head()

In [ ]:
dados = objeto.transform(dataframe)
dados_transformados = fit(df).transform(df)
dados_transformados = fit_transform(df)

In [ ]:
yeo_johnson = PowerTransformer(method = 'yeo-johnson', standardize = True)
A2_yeo_johnson = yeo_johnson.fit_transform(df_A2)
A2_yeo_johnson # array NumPy com os dados transformados

In [ ]:
df_A2_yeo_johnson = pd.DataFrame(A2_yeo_johnson, columns = ['coluna1', 'coluna2', 'coluna3', 'coluna4'])
df_A2_yeo_johnson.head()

In [ ]:
plt.figure(figsize = (12, 8))
plt.hist(df_A2_yeo_johnson['coluna3'], color = 'blue', edgecolor = 'black', bins = int(180/5))

# Adiciona títulos e labels
plt.title('Coluna3 - Distribuição aproximadamente Normal')

## Transformação de Box-Cox
* Inventada por dois grandes personagens da Estatística;
* A coluna/variável/atributo não pode conter números negativos ou zero. Ou seja, $x_{i} > 0$.

* Se $w_{i}$ é a variável transformada e $x_{i}$ é a variável que queremos transformar.
    * Se $\lambda = 0$ --> $w_{i}^{(\lambda)} = \log(x_{i})$;
    * Se $\lambda <> 0$ --> $w_{i}^{(\lambda)} = \frac{x_{i}-1}{\lambda}$;
* Se $\lambda = 1$, então $w_{i}$ então os dados/distribuição já são normalmente distribuídos e a transformação de Box&Cox não se faz necessário.
* Precisamos escolher o valor de $\lambda$ que permite a melhor aproximação da distribuição normal.
* A função scipy.stats.boxcox(array_1D) retorna o valor de $\lambda$ ótimo. Basta passar como parâmetro o array de dimensão 1D que a função retorna o $\lambda$ ótimo que melhor se ajusta aos seus dados.
* Para retornar seus dados aos valores originais, use scipy.special.inv_boxcox(y, lambda).
* Quais são as desvantagens/inconveniente da transformação?
    * Perde-se a interpretação.

Libraries necessárias:

In [ ]:
import numpy as np 
from scipy import stats 
import matplotlib.pyplot as plt 

In [ ]:
# Gerando dados com distribuição Exponencial
distribuicao_exponencial = np.random.exponential(size = 1000) 

# Dados transformados 
box_cox, lambda_box_cox = stats.boxcox(distribuicao_exponencial) 
f"lambda ótimo: {lambda_box_cox}"

In [ ]:
# Dados transformados pela Box-Cox:
box_cox[0:30]

### Exemplo 1
* Dados possuem distribuição Exponencial.

In [ ]:
# Gráficos: 
def compara_graficos(y, w, lambda_box_cox):
    fig, ax = plt.subplots(1, 2) 
  
    # Gráfico das distribuições originais e transformada
    sns.distplot(y, hist = False, kde = True, kde_kws = {'shade': True, 'linewidth': 2}, label = "Non-Normal", color = "green", ax = ax[0]) 
    sns.distplot(w, hist = False, kde = True, kde_kws = {'shade': True, 'linewidth': 2}, label = "Normal", color = "green", ax = ax[1]) 
  
    # Legendas 
    plt.legend(loc = "upper right")
  
    # Redimensionando os sub-gráficos 
    fig.set_figheight(5)
    fig.set_figwidth(10) 
    
    print(f"Valor de Lambda usado na transformação: {lambda_box_cox}") 

In [ ]:
compara_graficos(distribuicao_exponencial, box_cox, lambda_box_cox)

### Exemplo 2
* Dados possuem distribuição Beta.


In [ ]:
# Gerando dados com distribuição Exponencial
distribuicao_beta = np.random.beta(1, 3, 1000)

In [ ]:
# transform training data & save lambda value 
box_cox, lambda_box_cox = stats.boxcox(distribuicao_beta) 

In [ ]:
f"Lambda ótimo: {lambda_box_cox}"

In [ ]:
compara_graficos(distribuicao_beta, box_cox, lambda_box_cox)

### Transformação log
* De forma geral, a transformação **log** trata de dados skewed (diferentes da distribuição Normal), tornando os dados (ou a distribuição dos dados) mais "normal";
* Se os dados forem de alguma forma normalmente distribuídos, então nada muda.

In [ ]:
# Gerando dados com distribuição Exponencial
distribuicao_beta = np.random.beta(1, 3, 1000)

transformacao_log = np.log(distribuicao_beta)

# Aproveitando a função compara_graficos()
compara_graficos(distribuicao_beta, transformacao_log, 1)

___
# **Exercícios**
> Para cada um dos dataframes a seguir, aplique os seguintes steps:

* Padronizar o nome das colunas
    * Eliminar espaços entre os nomes das colunas;
    * Eliminar caracteres especiais dos nomes das colunas;
    * Renomear as colunas com lower() (ou upper());
* Aplicar a trasformação StandardScaler e MinMaxScaler em cada uma das colunas do dataframe;
* DataViz - Mostrar a distribuição das colunas para compararmos os resultados antes e depois das transformações.
* As correlações das colunas mudam com as transformações?

## Exercício 1 - Iris --> **Resolvido**
* [Aqui](https://en.wikipedia.org/wiki/Iris_flower_data_set) você obterá mais informações sobre o dataframe iris. Confira.

In [ ]:
from sklearn.datasets import load_iris

iris = load_iris()
X= iris['data']
y= iris['target']

df_iris = pd.DataFrame(np.c_[X, y], columns= np.append(iris['feature_names'], ['target']))
df_iris['target2'] = df_iris['target'].map({0: 'setosa', 1: 'versicolor', 2: 'virginica'})
df_iris.head()

In [ ]:
df_iris.columns = [c.replace(' ', '_') for c in df_iris.columns]
df_iris.columns = [c.replace('_(cm)', '') for c in df_iris.columns]
df_iris.head()

In [ ]:
df_iris.plot(kind = 'kde')

In [ ]:
# Aplica a transformação:
df_iris_MinMaxScaler = MinMaxScaler().fit_transform(df_iris[['sepal_length', 'sepal_width', 'petal_length', 'petal_width']])

# Transformando em Dataframe:
df_iris_MinMaxScaler = pd.DataFrame(df_iris_MinMaxScaler, columns = ['sepal_length', 'sepal_width', 'petal_length', 'petal_width'])

# Gráfico
df_iris_MinMaxScaler.plot(kind = 'kde')

### Aplicar as outras transformações e comparar os gráficos.

## Exercício 2 - Breast Cancer

In [ ]:
import pandas as pd
import numpy as np
from sklearn.datasets import load_breast_cancer

cancer = load_breast_cancer()
X= cancer['data']
y= cancer['target']

df_A1_cancer = pd.DataFrame(np.c_[X, y], columns= np.append(cancer['feature_names'], ['target']))
df_A1_cancer['target'] = df_A1_cancer['target'].map({0: 'malign', 1: 'benign'})
df_A1_cancer.head()

## Exercício 3 - Boston Housing Price

In [ ]:
from sklearn.datasets import load_boston

boston = load_boston()
X= boston['data']
y= boston['target']

df_A1_boston = pd.DataFrame(np.c_[X, y], columns= np.append(boston['feature_names'], ['target']))
df_A1_boston.head()

## Exercícios 4 - Diabetes

In [ ]:
from sklearn.datasets import load_diabetes

diabetes = load_diabetes()
X= diabetes['data']
y= diabetes['target']

df_A1_diabetes = pd.DataFrame(np.c_[X, y], columns= np.append(diabetes['feature_names'], ['target']))
df_A1_diabetes.head()

## Exercícios 5 - 120 years of Olympic history: athletes and results
* [120 years of Olympic history: athletes and results](https://www.kaggle.com/heesoo37/120-years-of-olympic-history-athletes-and-results)
    * Trate adequadamente as variáveis 'sex', 'season', 'team', 'city', 'sport' e 'medal';
    * Aplique as transformações que acabamos de estudar nos campos/colunas numéricas 'height' e 'weight'. Cuidado com os Missing Values contidos nas variáveis!
    * Verifique/avalie o impacto dos outliers nestas colunas.
    * Neste caso, qual transformação é mais adequado diante dos outliers?

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
url = '/content/drive/My Drive/Datasets4ML/athlete_events.csv'
df = pd.read_csv(url)
df.head()

## Exercício 6 - FIFA
* Aplique as transformações MinMaxScaler, RobustScaler e StandardScaler às colunas numéricas do dataframe FIFA_algumas_features.csv.
* Para as colunas categóricas, aplique a transformação mais adequada.

In [ ]:
import pandas as pd

In [ ]:
url = 'https://raw.githubusercontent.com/MathMachado/DataFrames/master/FIFA_algumas_features.csv?token=AGDJQ62CSW5KBLZNXH4TULK7SXICE'

df = pd.read_csv(url, index_col = 'ID')
df.head()

# WOE - Weight Of Evidence
* As vantagens da transformação WOE são
    * Lida bem com NaN's;
    * Lida bem com outliers;
    * A transformação é baseada no valor logarítmico das distribuições.
    * Usando a técnica de binning apropriada, pode estabelecer uma relação monotônica (aumentar ou diminuir) entre a variável dependente e independente.